In [1]:
'''手动读取.h5文件赋值给torch'''
import keras
from models.DFCAN16 import DFCAN as keras_DFCAN
from model.DFCAN import DFCAN as torch_DFCAN
input_height, input_width, n_channel = 502, 502, 1

weight_file = "model_weights.h5"

keras_model = keras_DFCAN((input_height, input_width, n_channel))
keras_model.load_weights(weight_file)

Using TensorFlow backend.
/root/miniconda3/envs/k2t_new/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#加载keras中可以学习的层的权重
keras_layers_name = []
keras_layers_weight = []
for layer in keras_model.layers:
    if layer.trainable == True:
        keras_layers_name.append(layer.name)
        keras_layers_weight.append(layer.weights)

In [3]:
#加载pytorch权重
torch_layers_names = []
torch_layers_params = []
model =  torch_DFCAN((n_channel))
for name, param in model.named_parameters():
    torch_layers_names.append(name)
    if param.requires_grad:
        # print(name, param.data)
        torch_layers_params.append(param)

In [4]:
import torch

In [5]:
#所有层的权重
for indx, weight in enumerate(keras_model.get_weights()):
    # weight = keras_model.get_weights()[0]   #keras第一层权重：numpy.ndarray
    if len(weight.shape) == 4:  #卷积核
        to_torch_weight = torch.Tensor(weight).permute(3,2,0,1) 
    elif len(weight.shape) == 1:  #偏置
        to_torch_weight = torch.Tensor(weight)
    else:
        raise BaseException
    # to_torch_weight = torch.Tensor(weight).permute(3,2,1,0)   
    [k for k in model.named_parameters()][indx][1].data = to_torch_weight
    # print(to_torch_weight[0,0,:,:])

In [6]:
# for indx, weight in enumerate(keras_model.get_weights()):
#     print(weight.shape)

In [7]:
#查看权重
indx = 12
print(keras_model.get_weights()[indx][:,:,0,0])    #keras卷积核
print([k for k in model.named_parameters()][indx][1][0,0,:,:].data)    #torch卷积核

[[ 0.01893339 -0.06204253 -0.02285613]
 [ 0.02468616  0.02675407 -0.00809869]
 [-0.02100416 -0.08708128 -0.04743621]]
tensor([[ 0.0189, -0.0620, -0.0229],
        [ 0.0247,  0.0268, -0.0081],
        [-0.0210, -0.0871, -0.0474]])


In [8]:
#生成固定输入
torch.manual_seed(123)
x0 = torch.randn(1, 1, 502, 502)

In [9]:
# #检查整个模型输出是否一致
# #pytorch
# torch_out = model(x0)

# #keras
# from keras import backend as K
# keras_x0 = x0.permute(0,2,3,1).numpy()
# print(type(keras_x0))
# print(keras_x0.shape)
# keras_output = keras_model.predict(keras_x0)
# print(type(keras_output))
# keras_output_torch_tensor = torch.Tensor(keras_output).permute(0, 3, 1, 2)

# #比较二者
# assert torch_out.shape == keras_output_torch_tensor.shape
# print(torch_out == keras_output_torch_tensor)
# print(torch_out.shape, keras_output_torch_tensor.shape)

In [10]:
# #查看一下卷积层输出
# print(torch_out[0,0,:3,:3])
# print(keras_output_torch_tensor[0,0,:3,:3])

In [11]:
#查看数据类型是否一致
# keras_model.get_weights()[indx][:,:,0,0].dtype
# x0.dtype
# [k for k in model.named_parameters()][indx][1][0,0,:,:].data.dtype
print(keras_model.get_weights()[indx][:,:,0,0], [k for k in model.named_parameters()][indx][1][0,0,:,:].data,sep="\n")
print(keras_model.get_weights()[indx][0,0,0,0], [k for k in model.named_parameters()][indx][1].detach().numpy()[0,0,0,0],sep="\n")
#一致，都是float32

[[ 0.01893339 -0.06204253 -0.02285613]
 [ 0.02468616  0.02675407 -0.00809869]
 [-0.02100416 -0.08708128 -0.04743621]]
tensor([[ 0.0189, -0.0620, -0.0229],
        [ 0.0247,  0.0268, -0.0081],
        [-0.0210, -0.0871, -0.0474]])
0.0189334
0.0189334


In [12]:
# #查看原始数据有没有出错
# print(x0[0,0,:3,:3])
# print(keras_x0[0,:3,:3,0])

In [13]:
#用官方的predict.py再跑跑看
#官方的predict.py和本文都用的predict，但是predict函数里面batchsize=32，也就是很可能不是直接取的模型输入

In [14]:
# #检查卷积层输出是否一致
# #pytorch
# torch_out = model.input[0](x0)

# #keras
# from keras import backend as K
# # keras_x0 = K.constant(x0.permute(0,2,3,1).numpy())  #转到keras的输入
# keras_x0 = x0.permute(0,2,3,1).numpy()
# # print(type(keras_x0))
# # print(keras_x0.shape)
# from keras.models import Model
# try:
#     layer_name = 'conv2d_84'
#     keras_model.get_layer(layer_name)
# except(ValueError):
#     layer_name = 'conv2d_1'
# intermediate_layer_model = Model(inputs=keras_model.input, outputs=keras_model.get_layer(layer_name).output)
# keras_output = intermediate_layer_model.predict(keras_x0)
# # print("keras卷积层输出：",type(keras_output))
# keras_output_torch_tensor = torch.Tensor(keras_output).permute(0, 3, 1, 2)

# #比较二者
# assert torch_out.shape == keras_output_torch_tensor.shape
# # print(torch_out == keras_output_torch_tensor)
# print("torch模型和keras模型卷积层输出shape：",torch_out.shape, keras_output_torch_tensor.shape)

In [15]:
#定义用于pytorch打印的hook函数
torch_inter_layer_out = None
def get_layer_output(layer, input, output):
    global torch_inter_layer_out
    torch_inter_layer_out = output

In [16]:
# #检查第二层输出是否一致

# #pytorch
# model.input.register_forward_hook(get_layer_output) #添加钩子
# model(x0)
# torch_out = torch_inter_layer_out

# #keras
# from keras import backend as K
# # keras_x0 = K.constant(x0.permute(0,2,3,1).numpy())  #转到keras的输入
# keras_x0 = x0.permute(0,2,3,1).numpy()
# # print(type(keras_x0))
# # print(keras_x0.shape)
# from keras.models import Model
# try:
#     layer_name = 'lambda_84'
#     keras_model.get_layer(layer_name)
# except(ValueError):
#     layer_name = 'lambda_1'
# intermediate_layer_model = Model(inputs=keras_model.input, outputs=keras_model.get_layer(layer_name).output)
# keras_output = intermediate_layer_model.predict(keras_x0)
# # print("keras卷积层输出：",type(keras_output))
# keras_output_torch_tensor = torch.Tensor(keras_output).permute(0, 3, 1, 2)

# #比较二者
# # assert torch_out.shape == keras_output_torch_tensor.shape
# # print(torch_out == keras_output_torch_tensor)
# print("torch模型和keras模型此层输出shape：",torch_out.shape, keras_output_torch_tensor.shape)
# print("torch模型和keras模型此层输出截取：",torch_out[0,0,:3,:3], keras_output_torch_tensor[0,0,:3,:3],sep="\n")

In [17]:
def get_k_layer_name(indx):
    #获取keras模型第k层的layer_name
    #获取keras_model所有层。实际上就是83个卷积层
    trainable_layer_names = [layer.name.split(r"/")[0] for layer in keras_model.trainable_weights]
    from collections import OrderedDict
    ordered_dict = OrderedDict.fromkeys(trainable_layer_names)
    set_trainable_layer_names = list(ordered_dict.keys())
    # set_trainable_layer_names
    # keras_model.get_layer(set_trainable_layer_names[0])
    return set_trainable_layer_names[indx]

In [30]:
#hook_layer
pre_index = 1
import re
s = torch_layers_names[pre_index*2]
s = re.sub(r'\.(\d+)\.', r'[\1].', s)
s = re.sub(r'\.(\d+)$', r'[\1]', s)
s = re.sub(r'\.[^.]*$', '', s)
print(s)

def get_member(cls, string):
    # 将输入字符串按照`.`分割成列表
    attribute_list = string.split(".")
    
    # 定义变量result，初始值为A
    result = cls
    
    # 遍历列表，访问对应的成员
    for attribute_name in attribute_list:
        if "[" in attribute_name and "]" in attribute_name:
            # 如果元素是以[数字]结尾的形式，就将数字提取出来，并使用getattr()函数访问result的对应成员
            index = int(attribute_name[attribute_name.index("[")+1:attribute_name.index("]")])
            attribute_name = attribute_name[:attribute_name.index("[")]
            result = getattr(result, attribute_name)[index]
        else:
            # 否则，直接使用getattr()函数访问result的对应成员
            result = getattr(result, attribute_name)
    return result

RGs[0].RCABs[0].conv_gelu1[0]


<bound method Module.parameters of Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))>

In [32]:
# 检查第layer_index层
layer_index = pre_index   #第3层
hook_layer = get_member(model,s)
hook_layer = model.RGs[0].RCABs[0].conv_gelu1[0]
#pytorch
hook_layer.register_forward_hook(get_layer_output) #添加钩子
model(x0)
torch_out = torch_inter_layer_out

#keras
from keras import backend as K
# keras_x0 = K.constant(x0.permute(0,2,3,1).numpy())  #转到keras的输入
keras_x0 = x0.permute(0,2,3,1).numpy()
# print(type(keras_x0))
# print(keras_x0.shape)
from keras.models import Model
layer_name = get_k_layer_name(layer_index)
intermediate_layer_model = Model(inputs=keras_model.input, outputs=keras_model.get_layer(layer_name).output)
keras_output = intermediate_layer_model.predict(keras_x0)
# print("keras卷积层输出：",type(keras_output))
keras_output_torch_tensor = torch.Tensor(keras_output).permute(0, 3, 1, 2)

#比较二者
# assert torch_out.shape == keras_output_torch_tensor.shape
# print(torch_out == keras_output_torch_tensor)
print("torch模型和keras模型此层输出shape：",torch_out.shape, keras_output_torch_tensor.shape)
print("torch模型和keras模型此层输出截取：",torch_out[0,0,:3,:3], keras_output_torch_tensor[0,0,:3,:3],sep="\n")

RuntimeError: Given groups=1, weight of size [64, 1, 3, 3], expected input[1, 64, 502, 502] to have 1 channels, but got 64 channels instead

In [20]:
# #查看临时模型权重和torch_model权重
# intermediate_layer_model.get_weights().__len__()
# print(intermediate_layer_model.get_weights()[4][:, :, 0, 0])
# print(torch_layers_params[4].data[0,0,:,:])

In [21]:
len(keras_model.layers)
list(model.modules()).__len__()
model.modules()

<generator object Module.modules at 0x7fb63445d780>

In [22]:
#单独看第三层
# layer_index = pre_index   #第3层

#构造输入
torch.manual_seed(123)
x0 = torch.randn((1, 64, 502, 502))

#pytorch
torch_out = model.RGs[0].RCABs[0].conv_gelu2[0](x0)

### keras
from keras import backend as K
from keras.layers import Input, Dense
from keras.layers.convolutional import Conv2D
from keras.models import Model

keras_x0 = x0.permute(0,2,3,1).numpy()
##单独第三层拿出来构建模型
#获取第3层
layers = keras_model.layers[5:6] #第3个卷积层
layer_config = [k.get_config() for k in layers]
layer_weights = [k.get_weights() for k in layers]
#构建单层模型
new_input = Input(shape=layer.input_shape[1:])
new_layer0 = Conv2D(**layer_config[0])(new_input)
new_layer0.se
# new_layer1 = Dense(**layer_config[0])(new_layer0)
# new_layer1.set_weights(layer_weights[0])
new_model = Model(new_input, new_layer0)
#拿来做预测
keras_output = new_model.predict(keras_x0)
# print("keras卷积层输出：",type(keras_output))
keras_output_torch_tensor = torch.Tensor(keras_output).permute(0, 3, 1, 2)

#比较二者
# assert torch_out.shape == keras_output_torch_tensor.shape
# print(torch_out == keras_output_torch_tensor)
print("torch模型和keras模型此层输出shape：",torch_out.shape, keras_output_torch_tensor.shape)
print("torch模型和keras模型此层输出截取：",torch_out[0,0,:3,:3], keras_output_torch_tensor[0,0,:3,:3],sep="\n")

AttributeError: 'Tensor' object has no attribute 'se'

In [ ]:
##可能是Torch和DDCAN的gelu定义不同，看看第1个卷积的定义

In [ ]:
layer = layers[0]
new_layer = Conv2D.from_config(layer.get_config())
new_layer.set_weights(layer.get_weights())

In [ ]:
from keras.layers import Conv2D
import numpy as np

# 假设我们有一个Conv2D层
conv_layer = Conv2D(filters=32, kernel_size=(3,3), input_shape=(28,28,1))

# 获取该层的配置和权重
layer_config = conv_layer.get_config()
layer_weights = conv_layer.get_weights()

# 创建一个新的Conv2D层
new_layer = Conv2D.from_config(layer_config)

# 设置新层的权重
new_layer.set_weights(layer_weights)

In [ ]:
type(conv_layer)
type(layers[0])
layers[0].get_config()

In [ ]:
torch_layers_names


In [ ]:
keras_model.layers[:10]


In [ ]:
print(keras_model.layers[5].get_weights()[0][:,:,0,0])
keras_model.get_weights()[4][:,:,0,0]


In [ ]:
print(torch_layers_names[:10])
print([get_k_layer_name(k) for k in range(10)])

In [ ]:
layer_name

In [ ]:
model.RGs[0].RCABs[0].conv_gelu2[0]

In [ ]:
keras_model.summary()

In [ ]:
# list(model.children())
print(list(model.named_parameters())[layer_index*2][1][0,0,:,:])
print(keras_model.get_weights()[layer_index*2][:,:,0,0])

# model._modules
# # model.RGs[0]


In [ ]:
print(torch_out[0,0,:3,:3], keras_output_torch_tensor[0,0,:3,:3],sep="\n")

In [ ]:
# intermediate_layer_model.summary()
print("临时keras卷积核\n",intermediate_layer_model.get_weights()[0][:,:,0,0])
print("临时keras卷积偏置", intermediate_layer_model.get_weights()[1][:3])
print("临时torch卷积核\n",[k for k in model.input[0].named_parameters()][0][1].data[0,0,:3,:3])
print("临时torch卷积偏置", torch_layers_params[1][:3])

In [ ]:
print(keras_model.get_weights()[0][:,:,0,0])
print(torch_layers_params[0][0,0,:,:].data)
torch_layers_params[0].shape
# keras_model.get_weights()[0].shape